# 📊 Analysis: Column Mapping Justification

This notebook explores the statistical and structural relationships between the columns to justify the inferred mapping.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load data and noise reduction
df = pd.read_csv("data.csv")
df.describe()

df_smoothed = df.copy()

columns_to_smooth = df.columns[:5]
span = min(1000, len(df) // 200) 

for column in columns_to_smooth:
    if df[column].dtype in ['int64', 'float64']:
        df_smoothed[column] = df[column].ewm(span=span, adjust=False).mean()
        print(f"Applied EWMA (span={span}) to column: {column}")

df_smoothed.to_csv('data_smoothed.csv', index=False)
print("Smoothed data saved to 'data_smoothed.csv'")

# comment the below line to remove noise reduction 
df = df_smoothed.copy() 


### 🔍 Step 1: Identifying Volume

In [ ]:
volume_counts = {}
columns = df.columns
for col in columns:
    volume_counts[col]=0

for index,row in df.iterrows():
    candidates = []
    for col in columns:
        value = row[col]
        if value > 0 and value == int(value):
            candidates.append(col)
    if candidates:
        max_value = 0
        best_col = None
        for col in candidates:
            if row[col] > max_value:
                max_value = row[col]
                best_col = col
        volume_counts[best_col] += 1

for col,count in volume_counts.items():
    volume_counts[col] = count / len(df)
max_column_volume = max(volume_counts, key=volume_counts.get)
max_confidence = volume_counts[max_column_volume]

plt.figure(figsize=(10, 6))
plt.bar(volume_counts.keys(), volume_counts.values())
plt.title('Probability of Each Column Being Volume')
plt.xlabel('Columns')
plt.ylabel('Probability')
plt.tight_layout()
plt.show()

print(f"Thus, Column {max_column_volume} is the most likely candidate for volume with confidence {max_confidence}")

### 🔍 Step 2: Correlation Analysis

In [ ]:
df.columns = df.columns.str.strip()

print("\n1. Pearson Correlation Matrix:")
pearson_corr = df.corr(method='pearson')
print(pearson_corr.round(5))

print("\n2. Spearman Correlation Matrix:")
spearman_corr = df.corr(method='spearman')
print(spearman_corr.round(5))

plt.figure(figsize=(15, 13))

plt.subplot(2, 2, 1)
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', center=0, square=True, fmt='.3f', cbar_kws={'label': 'Correlation'})
plt.title('Pearson Correlation Matrix')

plt.subplot(2, 2, 2)
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', center=0, square=True, fmt='.3f', cbar_kws={'label': 'Correlation'})
plt.title('Spearman Correlation Matrix')

### 🔍 Step 3: Row-wise Statistics inference (as applicable)

In [ ]:
filtered_columns = [col for col in columns if col != max_column_volume]
incomings = {col : 0 for col in filtered_columns}
outgoings = {col : 0 for col in filtered_columns}

for high_col in filtered_columns:
    for low_col in filtered_columns:
        if high_col == low_col:
            continue
            
        if (df[high_col] >= df[low_col]).all():
            print(f"{high_col} >= {low_col} for all rows")
            incomings[low_col] += 1
            outgoings[high_col] += 1

filtered_low = [col for col, value in incomings.items() if value == max(incomings.values())]
filtered_high = [col for col, value in outgoings.items() if value == max(outgoings.values())]


print(f"\nMost incoming column: {filtered_low} with {max(incomings.values())} incoming connections")
print(f"Most outgoing column: {filtered_high} with {max(outgoings.values()) } outgoing connections")

In [ ]:
filtered_columns = [col for col in columns if col != max_column_volume]

df_subset = df[filtered_columns]

max_indices = df_subset.values.argmax(axis=1)
min_indices = df_subset.values.argmin(axis=1)

max_cols = df_subset.columns[max_indices]
min_cols = df_subset.columns[min_indices]

larger_counts = max_cols.value_counts()
smaller_counts = min_cols.value_counts()

larger = {col: larger_counts.get(col, 0) for col in filtered_columns}
smaller = {col: smaller_counts.get(col, 0) for col in filtered_columns}

if 'pulse' in filtered_columns:
    pulse_high_mask = max_cols == 'pulse'
    pulse_low_mask = min_cols == 'pulse'
    
    vol_pulse_large = df.loc[pulse_high_mask, max_column_volume].sum()
    vol_pulse_small = df.loc[pulse_low_mask, max_column_volume].sum()
else:
    vol_pulse_large = 0
    vol_pulse_small = 0

for col in filtered_columns:
    print(f"Column {col} is largest {larger[col]} times and smallest {smaller[col]} times")

total_rows = len(df)
larger_prob = {col: count / total_rows for col, count in larger.items()}
smaller_prob = {col: count / total_rows for col, count in smaller.items()}

# Find columns with highest probabilities
max_col_high = max(larger_prob, key=larger_prob.get)
max_col_low = max(smaller_prob, key=smaller_prob.get)

large_price = larger_prob[max_col_high]
small_price = smaller_prob[max_col_low]

# Find another column with non-zero probability
max_col_price = None
for col in filtered_columns:
    if ((larger_prob[col] > 0 or smaller_prob[col] > 0) and 
        col not in (max_col_high, max_col_low)):
        max_col_price = col
        break

# Create plots
plt.figure(figsize=(10, 6))
plt.bar(larger_prob.keys(), larger_prob.values())
plt.title('Probability of Each Column Being High')
plt.xlabel('Columns')
plt.ylabel('Probability')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.bar(smaller_prob.keys(), smaller_prob.values())
plt.title('Probability of Each Column Being Low')
plt.xlabel('Columns')
plt.ylabel('Probability')
plt.tight_layout()
plt.show()

In [ ]:
columns = df.columns.tolist()
print("Columns:", columns)

df_prev = df.shift(1)

results = []

for open_col in columns:
    for close_col in columns:
        if open_col == close_col or open_col == max_col_price or close_col == max_col_price:
            continue
        
        diff_sum = abs(df[open_col].iloc[1:] - df_prev[close_col].iloc[1:]).sum()
        
        results.append((open_col, close_col, diff_sum))
        print(f"{open_col}, {close_col}, {diff_sum}")

min_result = min(results, key=lambda x: x[2])
print(f"\nLowest difference: {min_result[0]} (open) vs {min_result[1]} (close) = {min_result[2]}")

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

feature_columns = columns

y = df[max_col_price].values
X_full = df.drop(columns=[max_col_price]).values

model.fit(X_full, y)

y_pred_full = model.predict(X_full)
exact_matches = (abs(y - y_pred_full) < 0.01).sum()

print("Analyzing how dropping each feature affects the model")

base_score = model.score(X_full, y)
base_matches = exact_matches

print(f"Baseline model R² score: {base_score:.6f}")
print(f"Baseline exact matches: {base_matches}/{len(y)}")

for col_to_drop in feature_columns:
    if col_to_drop == max_col_price:
        print(f"\nSkipping target variable: {max_col_price}")
        continue
        
    X_reduced = df.drop(columns=[max_col_price, col_to_drop]).values
    
    loo_model = LinearRegression().fit(X_reduced, y)
    score = loo_model.score(X_reduced, y)
    
    loo_y_pred = loo_model.predict(X_reduced)
    loo_exact_matches = (abs(y - loo_y_pred) < 0.01).sum()
    
    # Print results
    print(f"\nDropped feature: {col_to_drop}")
    print(f"R² score: {score:.6f} (change: {score - base_score:+.6f})")
    print(f"Exact matches: {loo_exact_matches}/{len(y)} (change: {loo_exact_matches - base_matches:+d})")

In [ ]:
candidate_open_close = [col for col in filtered_columns if col not in {max_col_high, max_col_low, max_col_price}]
print(f"Candidate columns: {candidate_open_close}")

candidate_data = df[candidate_open_close].values
n_rows = len(df) - 1

results = []

for i, candidate_open in enumerate(candidate_open_close):
    for j, candidate_close in enumerate(candidate_open_close):
        if i == j:
            continue

        open_col = candidate_data[:, i]
        close_col = candidate_data[:, j]

        o1, o2 = open_col[:-1], open_col[1:]
        c1, c2 = close_col[:-1], close_col[1:]

        condition1 = (o1 > c1) & (o2 > c2) & (o1 > c2)
        condition2 = (o1 < c1) & (o2 < c2) & (o1 < c2)

        same = np.sum(condition1 | condition2)
        same_ratio = same / n_rows

        results.append({
            'open_col': candidate_open,
            'close_col': candidate_close,
            'same_absolute': same,
            'same_ratio': same_ratio
        })
        
        print(f"Analyzing open: {candidate_open} and close: {candidate_close}")
        print(f"Same absolute: {same}")
        print(f"Same ratio: {same_ratio:.4f}")
        print()



In [ ]:
size = len(df)

for i, candidate_open in enumerate(candidate_open_close):
    for j, candidate_close in enumerate(candidate_open_close):
        if i == j:
            continue
        
        works = 0
        
        open_data = df[candidate_open].values
        close_data = df[candidate_close].values
        
        for k in range(0, size, 1000):
            end_idx = min(k + 1000, size)
            chunk_end = min(k + 999, size - 1)
            
            total_diff = close_data[chunk_end] - open_data[k]
            
            chunk_open = open_data[k:end_idx]
            chunk_close = close_data[k:end_idx]
            candidate_sum = np.sum(chunk_close - chunk_open)
            
            if (candidate_sum > 0 and total_diff > 0) or (candidate_sum < 0 and total_diff < 0):
                works += 1
        
        print(f"Analyzing {candidate_open} -> {candidate_close}: {works}")


In [ ]:
print(candidate_open_close)

df.columns = df.columns.str.strip()
columns = df.columns.tolist()

print("Columns:", columns)

df_prev = df.shift(1)

results = []
best_open_percentage = float('inf')
max_open_col = None
max_close_col = None

for i, candidate_open in enumerate(candidate_open_close):
  for j, candidate_close in enumerate(candidate_open_close):
      if i == j:
          continue

      price_col = max_col_price

      closecount = 0
      opencount = 0
      
      open_distance = abs(df[price_col] - df_prev[candidate_open])
      close_distance = abs(df[price_col] - df_prev[candidate_close])
          
      closer_to_open = (open_distance < close_distance).sum()
      closer_to_close = (close_distance < open_distance).sum()
          
      opencount += closer_to_open
      closecount += closer_to_close
      
      open_percentage = 100 * opencount / (opencount + closecount)
      
      print(f"Open: {candidate_open}, Close: {candidate_close}, Price: {price_col}")
      print(f"Closer to Open: {opencount}")
      print(f"Closer to Close: {closecount}")
      print(f"Open % (lower the better): {open_percentage:.2f}")
      
      results.append({
          'open_col': candidate_open,
          'close_col': candidate_close,
          'open_percentage': open_percentage,
          'opencount': opencount,
          'closecount': closecount
    })
      
      if open_percentage < best_open_percentage:
          best_open_percentage = open_percentage
          max_open_col = candidate_open
          max_close_col = candidate_close

print(f"\nOptimal combination:")
print(f"max_open_col: {max_open_col}")
print(f"max_close_col: {max_close_col}")
print(f"Best open percentage: {best_open_percentage:.2f}%")

### 📈 Step 4: Candlestick Visual from Mapped Fields

In [ ]:
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpdates

plt.style.use('dark_background')

df['Date_num'] = df.index

ohlc_data = df[['Date_num', 'deltaX', 'gamma', 'omega', 'flux']].values

fig, ax = plt.subplots(figsize=(12, 6))

candlestick_ohlc(ax, ohlc_data, width=0.6, colorup='green', colordown='red', alpha=0.8)

ax.grid(True)
ax.set_xlabel('Index')
ax.set_ylabel('Price')
plt.title('Candlestick Chart (deltaX=Open, gamma=High, omega=Low, flux=Close)')


fig.tight_layout()
plt.show()

This confirms the structural pattern of price movement and supports the inferred mapping.